使用 Python abc 套件的原因，是為了要解決 Python 沒有「抽象類別 (abstract class)」的問題。透過抽象類別，我們可以建立一個比起使用 hasattr() 還要更嚴格的類別介面 (class interface) 檢查。

舉例而言，我們來建立一個動物的類別，我們希望之後繼承實作的類別都一定要有「screaming」以及「walk」的方法，我們可以透過 abc.ABCMeta 這個 metaclass 來實作 Animal 抽象類別

In [1]:
import abc
 
class Animal(metaclass=abc.ABCMeta):
 
    @abc.abstractmethod
    def screaming(self):
        'Return when animal screaming the sound hear likes'
        return NotImplemented
 
    @abc.abstractmethod
    def walk(self, x, y):
        'Make animal walk to position (x, y).'
        return NotImplemented

如果不透過 metaclass，我們也可以透過繼承 abc.ABC 這 helper class 達到相同的效果：

In [2]:
import abc
 
class Animal(abc.ABC):
 
    @abc.abstractmethod
    def screaming(self):
        'Return when animal screaming the sound hear likes'
        return NotImplemented
 
    @abc.abstractmethod
    def walk(self, x, y):
        'Make animal walk to position (x, y).'
        return NotImplemented


當我們使用 Animal 這個抽象類別來建立類別時，就必須要實作 screaming 以及 walk 。如果沒有實作的話，Python 就會產生 TypeError：

In [3]:
class Dog(Animal):
    pass

Dog()

TypeError: Can't instantiate abstract class Dog with abstract methods screaming, walk

In [5]:
# 實作Dog
class Dog(Animal):
    x = 0
    y = 0
    def screaming(self):
        return 'Wof, Wof'
    def walk(self, x, y):
        self.x = x
        self.y = y
        return (self.x, self.y)

dog = Dog()
dog.screaming()

'Wof, Wof'

In [8]:
# 查看類別
print(isinstance(Dog(), Animal))

print(issubclass(Dog, Animal))

True
True


前面提到了抽象方法，那其他的類別屬性也能夠抽象化嗎？答案是可以，自 Python 3.3 版後，只需要在 method decorator 後面加上 abstractmethod 即可：

In [9]:
import abc
 
class Base(abc.ABC):
 
    @classmethod
    @abc.abstractmethod
    def setUpClass(cls):
        return NotImplemented
 
    @staticmethod
    @abc.abstractmethod
    def count(self, data):
        return len(data)
 
 
class Implementation(Base):
 
    @classmethod
    def setUpClass(cls):
        cls.count = 0
 
    @staticmethod
    def count(self, data):
        self.count = len(data)
        return self.count

在 class 裡面的 property 也可以要求抽象化，同樣在 decorator 下面加上 abstractmethod 即可：

In [10]:
import abc
 
 
class Base(abc.ABC):
    _index = 0
 
    @property
    @abc.abstractmethod
    def index(self):
        return self._index
 
    @index.setter
    @abc.abstractmethod
    def index(self, new_index):
        self._index = new_index
 
 
class Implementation(Base):
    MAX_LEN = 100
 
    @property
    def index(self):
        return self._index
 
    @index.setter
    def index(self, new_index):
        new_index = min(new_index, self.MAX_LEN)
        self._index = new_index
 
imp = Implementation()
print(imp.index)
imp.index = 50
print(imp.index)
imp.index = 500
print(imp.index)

0
50
100


abc 的運作原理?

abc.ABCMeta 主要做到三件事情：

1. ABCMeta 是 metaclass，因此 override __new__ 方法，在這裡面會初始化 abstract 檢查的部分。
2. override __isinstancecheck__
3. override __issubclasscheck__